Firstly load all the required environments, be careful that the version of pytorch should be at least 2.0

In [1]:
import sys
sys.path.append('../')
from FEM import FEM
import torch

Then set up some hyperparameters that will be used in all problem instances.

In [2]:
num_trials = 500
num_steps = 1000
dev = 'cuda' # if you do not have gpu in your computing devices, then choose 'cpu' here

The first problem instance will be Maximum Cut (MaxCut) on Gset 1, the best known cut will be 11624.

In [4]:
case_maxcut = FEM.from_file(
    'maxcut', 'instances/G1.txt', index_start=1, discretization=True
)
case_maxcut.set_up_solver(
    num_trials, num_steps, manual_grad=True, betamin=0.001, betamax=0.5, 
    learning_rate=0.1, optimizer='rmsprop', dev=dev
)
config, result = case_maxcut.solve()
optimal_inds = torch.argwhere(result==result.max()).reshape(-1)
print(f'maxcut test instance, optimal value {result.max()}')

maxcut test instance, optimal value 11624.0


We can see that here the optimal value can be easily obtained by our FEM solver.The next problem instance will be balance minimum cut (bMinCut). The goal will be partition the graph into balanced piece and make sure the cut between them are as small as possible. Here we choose the well known karate club graph, and divide it into three pieces in order to demonstrate the ability of FEM to solve optimization problems with multiple state variables.

In [3]:
case_bmincut = FEM.from_file(
    'bmincut', 'instances/karate.txt', index_start=1
)
case_bmincut.set_up_solver(num_trials, num_steps, dev=dev, q=3)
config, result = case_bmincut.solve()
optimal_inds = torch.argwhere(result==result.min()).reshape(-1)
print(f'bmincut test instance, optimal value {result.min()}')

bmincut test instance, optimal value 20.0


The next problem instance is the famous vertex cover problem, the test graph is a small graph with 5 vertices and 6 edges. This graph is small enough thus we can explicitly show the optimal configuration here.

In [4]:
case_vertexcover = FEM.from_file(
    'vertexcover', 'instances/vertexcover.txt', index_start=0
)
case_vertexcover.set_up_solver(
    num_trials, num_steps, betamin=10, betamax=30, h_factor=1, dev=dev,
    learning_rate=0.01, anneal='exp'
)
config, result =case_vertexcover.solve()
optimal_inds = torch.argwhere(result==result.min()).reshape(-1)
print(f'vertexcover test instance, optimal value {result.min()}')
optimal_configs = torch.unique(config[optimal_inds], dim=0)
print('optimal configs are')
for conf in optimal_configs:
    print(conf.cpu().detach().numpy())

vertexcover test instance, optimal value 3.0
optimal configs are
[0. 1. 1. 0. 1.]
[0. 1. 1. 1. 0.]
[1. 0. 0. 1. 1.]
[1. 0. 1. 1. 0.]


The final build-in problem instances will be one with multi-variable interactions, which is unique from all problem instances above. It is the maximum k-satisfactory (Max-kSAT), which is given all the variable states, what is the maximum number of satisfied clauses. Here the test instance is from [Max-SAT 2016](http://www.maxsat.udl.cat/16/index.html).

In [4]:
case_maxksat = FEM.from_file(
    'maxksat', 'instances/s3v70c1000-1.cnf'
)
case_maxksat.set_up_solver(
    num_trials, num_steps, manual_grad=True, h_factor=0.3, anneal='lin',
    betamin=0.01, betamax=30, learning_rate=1.1, sparse=True, dev=dev,
)
config, result = case_maxksat.solve()
optimal_inds = torch.argwhere(result==result.min()).reshape(-1)
print(f'maxksat test instance, optimal value {result.min()}')

maxksat test instance, optimal value 47.0


Here FEM solver can find the optimal value as the best solver in the competition.